In [0]:
import torch as th

In [7]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [9]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [10]:
!pip install syft

  Using cached https://files.pythonhosted.org/packages/ef/49/1262a301e9db38f4822fa9e0e3b15831c86af9eaeb43b2b0aea3d0c6e1a1/syft-0.1.22a1-py3-none-any.whl
     |████████████████████████████████| 389kB 5.0MB/s 
     |████████████████████████████████| 81kB 26.7MB/s 
  Using cached https://files.pythonhosted.org/packages/07/ce/da9916e7e78f736894b15538b702c0b213fd5d60a7fd6e481d74033a90c0/tf_encrypted-0.5.6-py3-none-manylinux1_x86_64.whl
     |████████████████████████████████| 460kB 47.3MB/s 
     |████████████████████████████████| 256kB 46.5MB/s 
     |████████████████████████████████| 204kB 52.4MB/s 
     |████████████████████████████████| 266kB 58kB/s 
     |████████████████████████████████| 51kB 18.7MB/s 
     |████████████████████████████████| 122kB 46.9MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067093 sha256=929951ec7b1ad21791307ad0aa90baba754cd1073fb726a89fe3c4f88ec487bc
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24

In [11]:
import syft as sy

W0806 13:56:46.161303 140453661996928 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0806 13:56:46.180453 140453661996928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [13]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

**Lesson : Basic Remote Execution in PySyft**

In [0]:
bob = sy.VirtualWorker(hook, id='bob')

In [15]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [18]:
bob._objects

{43998774710: tensor([1, 2, 3, 4, 5])}

In [19]:
x.location

<VirtualWorker id:bob #objects:1>

In [20]:
x.id_at_location

43998774710

In [21]:
x.id

96665505889

In [22]:
x.owner

<VirtualWorker id:me #objects:0>

In [23]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [24]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [25]:
bob._objects

{}

**Project: Playing with Remote Tensors**

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [29]:
x_ptr.child.child

{'alice': [PointerTensor | me:15865047686 -> alice:78660278088],
 'bob': [PointerTensor | me:7340719419 -> bob:2565304846]}

In [30]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [32]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

**Lesson: Introducing Remote Arithmetic**

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [34]:
x

(Wrapper)>[PointerTensor | me:58084396986 -> bob:95779312738]

In [35]:
y

(Wrapper)>[PointerTensor | me:29930823926 -> bob:21384714613]

In [0]:
z = x + y

In [37]:
z

(Wrapper)>[PointerTensor | me:14130850944 -> bob:79574593385]

In [38]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [39]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:28111432779 -> bob:84313606654]

In [40]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x+y).sum()

In [43]:
z.backward()

(Wrapper)>[PointerTensor | me:37165357306 -> bob:34896268191]

In [0]:
x = x.get()

In [45]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [46]:
x.grad

tensor([1., 1., 1., 1., 1.])

**Project: Learn a Simple Linear Model**

In [0]:
# make a toy dataset
input = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [49]:
for i in range(10):
  
  pred = input.mm(weights)

  loss = ((pred - target)**2).sum()

  loss.backward()

  weights.data.sub_(weights.grad * 0.1)
  weights.grad *= 0

  print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


**Lesson: Garbage Collection and Common Errors**

In [0]:
bob = bob.clear_objects()

In [51]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{62382151461: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [55]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [57]:
x

(Wrapper)>[PointerTensor | me:86816378614 -> bob:13951627005]

In [58]:
bob._objects

{13951627005: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [60]:
bob._objects

{13951627005: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [62]:
bob._objects

{13951627005: tensor([1, 2, 3, 4, 5])}

In [63]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,2,3,4,5])

In [0]:
z = x + y

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,2,3,4,5]).send(alice)

In [67]:
z = x + y

TensorsNotCollocatedException: ignored

**Lesson: Toy Federated Learning**

In [0]:
from torch import nn, optim

In [0]:
# A toy dataset
data = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [0]:
# A Toy model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
def train(iterations=20):
  for iter in range(20):
    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()
    opt.step()
    print(loss.data)

In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):
  model = nn.Linear(2,1)
  opt = optim.SGD(params=model.parameters(), lr=0.1)
  
  for iter in range(iterations):
    
    for _data, _target in datasets:
      
      #send model to the data
      model = model.send(_data.location)
      
      # do normal training
      opt.zero_grad()
      pred = model(_data)
      loss = ((pred - _target)**2).sum()
      loss.backward()
      opt.step()
      
      # get smarter model back
      model = model.get()
      
      print(loss.get())

In [77]:
train()

tensor(1.0269, requires_grad=True)
tensor(0.0717, requires_grad=True)
tensor(0.0537, requires_grad=True)
tensor(0.0446, requires_grad=True)
tensor(0.0371, requires_grad=True)
tensor(0.0309, requires_grad=True)
tensor(0.0257, requires_grad=True)
tensor(0.0214, requires_grad=True)
tensor(0.0178, requires_grad=True)
tensor(0.0148, requires_grad=True)
tensor(0.0123, requires_grad=True)
tensor(0.0103, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0071, requires_grad=True)
tensor(0.0059, requires_grad=True)
tensor(0.0049, requires_grad=True)
tensor(0.0041, requires_grad=True)
tensor(0.0034, requires_grad=True)
tensor(0.0028, requires_grad=True)
tensor(0.0024, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0016, requires_grad=True)
tensor(0.0014, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0009, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0005, requi

**Lesson :** Advanced Remote Execution Tools

In [95]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [96]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [98]:
x.send(alice)

(Wrapper)>[PointerTensor | me:125737944 -> alice:34679679524]

In [99]:
bob._objects

{43677048617: tensor([1, 2, 3, 4, 5])}

In [100]:
alice._objects

{34679679524: (Wrapper)>[PointerTensor | alice:34679679524 -> bob:43677048617]}

In [0]:
y = x + x

In [102]:
y

(Wrapper)>[PointerTensor | me:41476035765 -> bob:54227271707]

In [103]:
bob._objects

{43677048617: tensor([1, 2, 3, 4, 5]),
 54227271707: tensor([ 2,  4,  6,  8, 10])}

In [104]:
alice._objects

{34679679524: (Wrapper)>[PointerTensor | alice:34679679524 -> bob:43677048617]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
# y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [112]:
bob._objects

{90195200451: tensor([1, 2, 3, 4, 5])}

In [113]:
alice._objects

{54165771993: (Wrapper)>[PointerTensor | alice:54165771993 -> bob:90195200451]}

In [114]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:54165771993 -> bob:90195200451]

In [115]:
bob._objects

{90195200451: tensor([1, 2, 3, 4, 5])}

In [116]:
alice._objects

{}

In [117]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [118]:
alice._objects

{}

In [119]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [121]:
bob._objects

{40708945831: tensor([1, 2, 3, 4, 5])}

In [122]:
alice._objects

{10801929412: (Wrapper)>[PointerTensor | alice:10801929412 -> bob:40708945831]}

In [0]:
del x

In [124]:
bob._objects

{}

In [125]:
alice._objects

{}

**Lesson:** Pointer Chain Operations

In [126]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [128]:
bob._objects

{22431223411: tensor([1, 2, 3, 4, 5])}

In [129]:
alice._objects

{23318366738: (Wrapper)>[PointerTensor | alice:23318366738 -> bob:22431223411]}

In [0]:
x = x.remote_get()

In [131]:
bob._objects

{}

In [132]:
alice._objects

{23318366738: tensor([1, 2, 3, 4, 5])}

In [133]:
x.move(bob)

(Wrapper)>[PointerTensor | me:59297662644 -> bob:9262286171]

In [134]:
x

(Wrapper)>[PointerTensor | me:59297662644 -> bob:9262286171]

In [135]:
bob._objects

{9262286171: tensor([1, 2, 3, 4, 5])}

In [136]:
alice._objects

{}